# TOD Simulation for schools 

Note: All @orca elements are in the scrpits folder. They should be the same as the ones prentented here. 

## Importing Libraries

In [4]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
from collections import OrderedDict
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca
# import os; os.chdir('/home/juan/activitysynth/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from sklearn.neighbors import BallTree

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs
import math 
from collections import OrderedDict 

import pickle
import dill
import time
import random

import urbansim_templates

from scripts import datasources, models, variables, utils

## Preparing data for simulation

In [5]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [6]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [7]:
orca.run(['initialize_network_small']) 
orca.run([ 'initialize_network_walk']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


## Preparing data for simulation 
This should be removed when SmallStep templates works

In [32]:
@orca.table(cache= True)
def TOD_school_data_preparation(students):
    """
    Transforms students data from wide to long format for pylogit prediction
    """
    students = orca.get_table('students').to_frame()
    
    #Creating a fake alternative specific varible and its corresponding availability variable. 
    for x in [1,2,3,4,5]:
        name = 'tt_'+str(x)
        av_name = 'av_'+str(x)
        students[name] = [random.randint(1,20) for x in range(len(students))]
        students[av_name] = 1
        students['choice'] = [random.randint(1,5) for x in range(len(students))]
    students.reset_index(inplace = True)
    
    #Preparing for wide to long format transformation
    ind_variables = ['age_0_5', 'age_5_12','age_12_15','age_15_18', 
                     'minority', 'asian', 'black','hh_inc_under_25k',
                     'hh_inc_25_to_75k', 'hh_inc_75_to_200k', 
                     'female', 'hh_size_over_4']

    alt_varying_variables = {u'travel_time': dict([(1, 'tt_1'),
                                               (2, 'tt_2'),
                                               (3, 'tt_3'),
                                               (4, 'tt_4'),
                                               (5, 'tt_5')])}

    availability_variables = dict(zip(range(1, 6), ['av_1','av_2','av_3','av_4','av_5']))
    
    # Perform the desired conversion
    df = pl.convert_wide_to_long(wide_data= students,
                                       ind_vars = ind_variables,
                                       alt_specific_vars = alt_varying_variables,
                                       availability_vars = availability_variables, 
                                       obs_id_col = 'person_id',
                                       choice_col= 'choice')  
    return df

In [38]:
@orca.step()
def TOD_school_arrival_simulate(persons):
    """
    Generate school arrival time of the day category.
    Category 1: 
    Category 2:
    Category 3:
    Category 4:
    Category 5:
    
    Note: Prediction is done using pylogit library directly. Data preparation is intended to transform from 
    wide format to long format. 
    """
    #Running the simulation
    file_Name = "/home/juan/activitysynth/activitysynth/configs/TOD_school_arrival.pkl"
    fileObject = open(file_Name,'rb')  
    TOD_arrival = pickle.load(fileObject)
    data = orca.get_table('TOD_school_data_preparation').to_frame()
    data['probabilities']= TOD_arrival.predict(data)
    TOD_choice = data.sort_values("probabilities", ascending=False).groupby('person_id').agg({'alt_id': 'first'})

    #Adding column to persons table
    persons.update_col('TOD_school_arrival', TOD_choice)

In [39]:
@orca.step()
def TOD_school_departure_simulate(persons):
    """
    Generate school departure time of the day category.
    Category 1: 
    Category 2:
    Category 3:
    Category 4:
    Category 5:
    
    Note: Prediction is done using pylogit library directly. Data preparation is intended to transform from 
    wide format to long format. 
    """
    #Running the simulation
    file_Name = "/home/juan/activitysynth/activitysynth/configs/TOD_school_departure.pkl"
    fileObject = open(file_Name,'rb')  
    TOD_departure = pickle.load(fileObject)
    data = orca.get_table('TOD_school_data_preparation').to_frame()
    data['probabilities']= TOD_departure.predict(data)
    TOD_choice = data.sort_values("probabilities", ascending=False).groupby('person_id').agg({'alt_id': 'first'})

    #Adding column to persons table
    persons.update_col('TOD_school_departure', TOD_choice)

In [40]:
orca.run([ 'TOD_school_arrival_simulate','TOD_school_departure_simulate' ]) 

Running step 'TOD_school_arrival_simulate'
Time to execute step 'TOD_school_arrival_simulate': 13.63 s
Running step 'TOD_school_departure_simulate'
Time to execute step 'TOD_school_departure_simulate': 13.82 s
Total time to execute iteration 1 with iteration value None: 27.45 s
